In [14]:
library(data.table)
# Versión mínima
archivos <- list.files(
  path = "./preds/",  # o la ruta correcta
  pattern = "prediccion_prod_completa_.*\\.txt$",
  full.names = TRUE  # para obtener rutas completas
)

if(length(archivos) == 0) stop("No hay archivos de predicciones")

lista_preds <- lapply(archivos, function(archivo) {
  tb <- fread(archivo, sep = "\t")
  setorder(tb, numero_de_cliente)  # Ordenar siempre por cliente
  return(tb)
})

# Extraer clientes ordenados
clientes_ordenados <- lista_preds[[1]]$numero_de_cliente

# Verificar que todos tengan los mismos clientes (ahora ordenados igual)
for(i in 2:length(lista_preds)) {
  if(!identical(clientes_ordenados, lista_preds[[i]]$numero_de_cliente)) {
    cat("❌ Diferentes clientes en archivo:", archivos[i], "\n")
    # Continuar de todos modos si solo hay diferencias menores
  }
}

# Calcular promedio
matriz_probs <- sapply(lista_preds, function(x) x$prob)
vprob_promedio <- rowMeans(matriz_probs)

# Crear output ordenado
tb_final <- data.table(numero_de_cliente = clientes_base, prob = vprob_promedio)
setorder(tb_final, -prob)
tb_final[1:11000, Predicted := 1L]
tb_final[is.na(Predicted), Predicted := 0L]

fwrite(tb_final[Predicted == 1L, .(numero_de_cliente)], 
       "submission3.csv", sep = ",", col.names = FALSE)

In [ ]:
getwd()


[1] "/Users/manumoreira/Repos/dmeyf2025/competencia3/notebooks/prod"